## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,66.04,76,0,6.42,clear sky
1,Ponta Delgada,PT,37.7333,-25.6667,65.50,72,20,14.97,few clouds
2,Tripoli,LY,32.8752,13.1875,62.73,47,90,5.44,overcast clouds
3,Ushuaia,AR,-54.8000,-68.3000,64.02,39,40,12.66,scattered clouds
4,Los Llanos De Aridane,ES,28.6585,-17.9182,66.13,79,20,3.00,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,66.04,76,0,6.42,clear sky
1,Ponta Delgada,PT,37.7333,-25.6667,65.50,72,20,14.97,few clouds
2,Tripoli,LY,32.8752,13.1875,62.73,47,90,5.44,overcast clouds
3,Ushuaia,AR,-54.8000,-68.3000,64.02,39,40,12.66,scattered clouds
4,Los Llanos De Aridane,ES,28.6585,-17.9182,66.13,79,20,3.00,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   501
Country                496
Lat                    501
Lng                    501
Max Temp               501
Humidity               501
Cloudiness             501
Wind Speed             501
Current Description    501
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,66.04,76,0,6.42,clear sky
1,Ponta Delgada,PT,37.7333,-25.6667,65.50,72,20,14.97,few clouds
2,Tripoli,LY,32.8752,13.1875,62.73,47,90,5.44,overcast clouds
3,Ushuaia,AR,-54.8000,-68.3000,64.02,39,40,12.66,scattered clouds
4,Los Llanos De Aridane,ES,28.6585,-17.9182,66.13,79,20,3.00,few clouds
...,...,...,...,...,...,...,...,...,...
685,Tecoanapa,MX,16.5167,-98.7500,82.35,50,1,7.99,clear sky
687,Alyangula,AU,-13.8483,136.4192,78.94,83,92,8.79,overcast clouds
688,Moses Lake,US,47.1301,-119.2781,39.24,93,100,6.91,fog
691,Buraydah,SA,26.3260,43.9750,56.75,29,0,7.52,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.04,clear sky,-33.6500,115.3333,
1,Ponta Delgada,PT,65.50,few clouds,37.7333,-25.6667,
2,Tripoli,LY,62.73,overcast clouds,32.8752,13.1875,
3,Ushuaia,AR,64.02,scattered clouds,-54.8000,-68.3000,
4,Los Llanos De Aridane,ES,66.13,few clouds,28.6585,-17.9182,
5,Hobart,AU,59.02,broken clouds,-42.8794,147.3294,
6,Kapaa,US,64.72,scattered clouds,22.0752,-159.3190,
7,Tura,IN,50.34,few clouds,25.5198,90.2201,
8,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,
10,Rikitea,PF,78.64,scattered clouds,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Create counters.
record_count = 1
set_count = 1
        
#6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    if record_count % 50 == 0 and record_count >= 50:
        record_count = 1
        set_count += 1
        time.sleep(60)
                
    record_count += 1
    print(f"Processing Record {record_count} of Set {set_count}")
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Processing Record 2 of Set 1
Processing Record 3 of Set 1
Processing Record 4 of Set 1
Processing Record 5 of Set 1
Processing Record 6 of Set 1
Processing Record 7 of Set 1
Processing Record 8 of Set 1
Processing Record 9 of Set 1
Processing Record 10 of Set 1
Processing Record 11 of Set 1
Processing Record 12 of Set 1
Processing Record 13 of Set 1
Processing Record 14 of Set 1
Processing Record 15 of Set 1
Processing Record 16 of Set 1
Processing Record 17 of Set 1
Processing Record 18 of Set 1
Processing Record 19 of Set 1
Processing Record 20 of Set 1
Processing Record 21 of Set 1
Processing Record 22 of Set 1
Processing Record 23 of Set 1
Processing Record 24 of Set 1
Processing Record 25 of Set 1
Processing Record 26 of Set 1
Processing Record 27 of Set 1
Processing Record 28 of Set 1
Processing Record 29 of Set 1
Processing Record 30 of Set 1
Processing Record 31 of Set 1
Processing Record 32 of Set 1
Processing Record 33 of Set 1
Processing Record 34 of Set 1
Processing Record 

Processing Record 20 of Set 6
Processing Record 21 of Set 6
Processing Record 22 of Set 6
Processing Record 23 of Set 6
Processing Record 24 of Set 6
Processing Record 25 of Set 6
Processing Record 26 of Set 6
Processing Record 27 of Set 6
Processing Record 28 of Set 6
Processing Record 29 of Set 6
Processing Record 30 of Set 6
Processing Record 31 of Set 6
Processing Record 32 of Set 6
Processing Record 33 of Set 6
Processing Record 34 of Set 6
Processing Record 35 of Set 6
Processing Record 36 of Set 6
Processing Record 37 of Set 6
Processing Record 38 of Set 6
Processing Record 39 of Set 6
Processing Record 40 of Set 6
Processing Record 41 of Set 6
Processing Record 42 of Set 6
Processing Record 43 of Set 6
Processing Record 44 of Set 6
Processing Record 45 of Set 6
Processing Record 46 of Set 6
Processing Record 47 of Set 6
Processing Record 48 of Set 6
Processing Record 49 of Set 6
Processing Record 50 of Set 6
Processing Record 2 of Set 7
Processing Record 3 of Set 7
Processing R

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.04,clear sky,-33.6500,115.3333,Observatory Guest House
1,Ponta Delgada,PT,65.50,few clouds,37.7333,-25.6667,Vila Nova Hotel
2,Tripoli,LY,62.73,overcast clouds,32.8752,13.1875,Hotel Al-Safwa
3,Ushuaia,AR,64.02,scattered clouds,-54.8000,-68.3000,Albatross Hotel
4,Los Llanos De Aridane,ES,66.13,few clouds,28.6585,-17.9182,Valle Aridane


In [21]:
# 8a. Create the output File (CSV)

#"WeatherPy_vacation.csv" = clean_hotel_df
output_data_file = clean_hotel_df

# # 8b. Export the City_Data into a csv
# clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))